In [31]:
import glob
import pydicom as pdi
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
def dicom2ndarray(dicom):
    img = pdi.read_file(dicom).pixel_array
    return img

def dicom2img(dicom):
    img = dicom2ndarray(dicom)
    plt.figure(figsize=(16,8))
    plt.title(dicom)
    plt.imshow(img, cmap='gray')

In [82]:
dicom_path_notebook = r'../../Dataset/CBIS_DDSM/*/*/*/*.dcm'

full_mammogram_dicom = r'../../Dataset/CBIS_DDSM/*/*/*full*/1-1.dcm'
roi_mammogram_dicom = r'../../Dataset/CBIS_DDSM/*/*/*ROI*/1-1.dcm'
mask_mammogram_dicom = r'../../Dataset/CBIS_DDSM/*/*/*ROI*/1-2.dcm'

calc_test_description = r'../../Dataset/CBIS_DDSM/calc_case_description_test_set.csv'

full_mammogram_path = glob.glob(full_mammogram_dicom)
roi_mammogram_path  = glob.glob(roi_mammogram_dicom)
mask_mammogram_path  = glob.glob(mask_mammogram_dicom)
calc_test_csv = glob.glob(calc_test_description)

In [83]:
calc_test_csv

['../../Dataset/CBIS_DDSM/calc_case_description_test_set.csv']

In [51]:
# print(full_mammogram_path)
# print(roi_mammogram_path)
# print(mask_mammogram_path)

['../../Dataset/CBIS_DDSM\\Calc-Test_P_00038_RIGHT_CC\\08-29-2017-DDSM-28468\\1.000000-full mammogram images-63304\\1-1.dcm', '../../Dataset/CBIS_DDSM\\Calc-Test_P_00038_RIGHT_MLO\\08-29-2017-DDSM-85215\\1.000000-full mammogram images-08846\\1-1.dcm', '../../Dataset/CBIS_DDSM\\Calc-Test_P_00041_LEFT_CC\\08-29-2017-DDSM-52275\\1.000000-full mammogram images-92812\\1-1.dcm']
['../../Dataset/CBIS_DDSM\\Calc-Test_P_00038_RIGHT_CC_1\\08-29-2017-DDSM-83105\\1.000000-ROI mask images-37851\\1-1.dcm', '../../Dataset/CBIS_DDSM\\Calc-Test_P_00038_RIGHT_CC_2\\08-29-2017-DDSM-67143\\1.000000-ROI mask images-07570\\1-1.dcm', '../../Dataset/CBIS_DDSM\\Calc-Test_P_00038_RIGHT_MLO_1\\08-29-2017-DDSM-65172\\1.000000-ROI mask images-21714\\1-1.dcm', '../../Dataset/CBIS_DDSM\\Calc-Test_P_00038_RIGHT_MLO_2\\08-29-2017-DDSM-68463\\1.000000-ROI mask images-04551\\1-1.dcm', '../../Dataset/CBIS_DDSM\\Calc-Test_P_00041_LEFT_CC_2\\08-29-2017-DDSM-12775\\1.000000-ROI mask images-79220\\1-1.dcm']
['../../Dataset/C

In [134]:
# csv에서 roi mask의 ID들만 추출
# 각 csv파일당 한번만 하면됨

csv_df = pd.read_csv(calc_test_csv[0])
ID_list = []
for i in (csv_df['ROI mask file path']):
    ID_list.append(i.split('/')[0])

ID_ndr = np.array(ID_list)


In [162]:
path = mask_mammogram_path[0]
ID = mask_mammogram_path[0].split("\\")[1]

# csv내의 ID(ROI mask file path)와 Dicom파일의 ID를 매칭
# np.argwhere 사용
idx = np.argwhere(ID_ndr == ID)
idx = idx[0][0]
idx

abnormal_type = csv_df.loc[idx]["abnormality type"]
pathology = csv_df.loc[idx]["pathology"]

# dicom file 내의 pixel data내에서 x,y,w,h 추출
img = dicom2ndarray(mask_mammogram_path[0])

test_info = cv.findNonZero(img)

min_y = test_info[...,0].min()
max_y = test_info[...,0].max()

min_x = test_info[...,1].min()
max_x = test_info[...,1].max()

minLoc = (min_y,min_x)
maxLoc = (max_y,max_x)

x = min_x
y = min_y
width = max_x - min_x
height = max_y - min_y


full_info_dict = {"path" : path, "ID" : ID, "abnormal_type":abnormal_type, "pathology":pathology, "x":x, "y":y, "width":width, "height":height}
full_info_list = [path, ID, abnormal_type, pathology, x, y, width, height]

In [165]:
full_info_list

['../../Dataset/CBIS_DDSM\\Calc-Test_P_00038_RIGHT_CC_1\\08-29-2017-DDSM-83105\\1.000000-ROI mask images-37851\\1-2.dcm',
 'Calc-Test_P_00038_RIGHT_CC_1',
 'calcification',
 'BENIGN_WITHOUT_CALLBACK',
 1024,
 2104,
 368,
 632]

In [198]:
#df = pd.DataFrame(index=["path", "ID", "abnormal_type", "pathology", "x", "y", "width", "height"])
df = pd.DataFrame(columns=["path", "ID", "abnormal_type", "pathology", "x", "y", "width", "height"])
df = pd.concat([df,full_info_dict])
df

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [202]:
pd.DataFrame(full_info_dict,columns=["path", "ID", "abnormal_type", "pathology", "x", "y", "width", "height"])

ValueError: If using all scalar values, you must pass an index